<a href="https://colab.research.google.com/github/vinay10949/AnalyticsAndML/blob/master/2_1_One_hot_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## One Hot Encoding

One hot encoding, consists in encoding each categorical variable with different boolean variables (also called dummy variables) which take values 0 or 1, indicating if a category is present in an observation.

For example, for the categorical variable "Gender", with labels 'female' and 'male', we can generate the boolean variable "female", which takes 1 if the person is 'female' or 0 otherwise, or we can generate the variable "male", which takes 1 if the person is 'male' and 0 otherwise.

For the categorical variable "colour" with values 'red', 'blue' and 'green', we can create 3 new variables called "red", "blue" and "green". These variables will take the value 1, if the observation is of the said colour or 0 otherwise. 


### Encoding into k-1 dummy variables

Note however, that for the variable "colour", by creating 2 binary variables, say "red" and "blue", we already encode **ALL** the information:

- if the observation is red, it will be captured by the variable "red" (red = 1, blue = 0)
- if the observation is blue, it will be captured by the variable "blue" (red = 0, blue = 1)
- if the observation is green, it will be captured by the combination of "red" and "blue" (red = 0, blue = 0)

We do not need to add a third variable "green" to capture that the observation is green.

More generally, a categorical variable should be encoded by creating k-1 binary variables, where k is the number of distinct categories. In the case of gender, k=2 (male / female), therefore we need to create only 1 (k - 1 = 1) binary variable. In the case of colour, which has 3 different categories (k=3), we need to create 2 (k - 1 = 2) binary variables to capture all the information.

One hot encoding into k-1 binary variables takes into account that we can use 1 less dimension and still represent the whole information: if the observation is 0 in all the binary variables, then it must be 1 in the final (not present) binary variable.

**When one hot encoding categorical variables, we create k - 1 binary variables**


Most machine learning algorithms, consider the entire data set while being fit. Therefore, encoding categorical variables into k - 1 binary variables, is better, as it avoids introducing redundant information.


### Exception: One hot encoding into k dummy variables

There are a few occasions when it is better to encode variables into k dummy variables:

- when building tree based algorithms
- when doing feature selection by recursive algorithms
- when interested in determine the importance of each single category

Tree based algorithms, as opposed to the majority of machine learning algorithms, **do not** evaluate the entire dataset while being trained. They randomly extract a subset of features from the data set at each node for each tree. Therefore, if we want a tree based algorithm to consider **all** the categories, we need to encode categorical variables into **k binary variables**.

If we are planning to do feature selection by recursive elimination (or addition), or if we want to evaluate the importance of each single category of the categorical variable, then we will also need the entire set of binary variables (k) to let the machine learning model select which ones have the most predictive power.


### Advantages of one hot encoding

- Straightforward to implement
- Makes no assumption about the distribution or categories of the categorical variable
- Keeps all the information of the categorical variable
- Suitable for linear models

### Limitations

- Expands the feature space
- Does not add extra information while encoding
- Many dummy variables may be identical, introducing redundant information


### Notes

If our datasets contain a few highly cardinal variables, we will end up very soon with datasets with thousands of columns, which may make training of our algorithms slow, and model interpretation hard.

In addition, many of these dummy variables may be similar to each other, since it is not unusual that 2 or more variables share the same combinations of 1 and 0s. Therefore one hot encoding may introduce redundant or duplicated information even if we encode into k-1.



In [4]:
!pip install feature_engine
import pandas as pd

# to split the datasets
from sklearn.model_selection import train_test_split

# for one hot encoding with sklearn
from sklearn.preprocessing import OneHotEncoder

# for one hot encoding with feature-engine
from feature_engine.categorical_encoders import OneHotCategoricalEncoder

  Created wheel for numpydoc: filename=numpydoc-0.9.2-cp36-none-any.whl size=31893 sha256=8988a3de7c126e6831b8a3ff940f17bd23c9ea8e299336574041d8651a1bde6c
  Stored in directory: /root/.cache/pip/wheels/96/f3/52/25c8e1f40637661d27feebc61dae16b84c7cdd93b8bc3d7486
Successfully built numpydoc


In [7]:
# load titanic dataset

data = pd.read_csv('titanic_train.csv',
                   usecols=['Sex', 'Embarked', 'Cabin', 'Survived'])
data.head()

,Survived,Sex,Cabin,Embarked
0,0,male,NaN,S
1,1,female,C85,C
2,1,female,NaN,S
3,1,female,C123,S
4,0,male,NaN,S


In [9]:
# let's capture only the first letter of the 
# cabin for this demonstration

data['Cabin'] = data['Cabin'].str[0]

data.head()

,Survived,Sex,Cabin,Embarked
0,0,male,NaN,S
1,1,female,C,C
2,1,female,NaN,S
3,1,female,C,S
4,0,male,NaN,S


### Encoding important

Just like imputation, all methods of categorical encoding should be performed over the training set, and then propagated to the test set. 

Why? 

Because these methods will "learn" patterns from the train data, and therefore you want to avoid leaking information and overfitting. But more importantly, because we don't know whether in future / live data, we will have all the categories present in the train data, or if there will be more or less categories. Therefore, we want to anticipate this uncertainty by setting the right processes right from the start. We want to create transformers that learn the categories from the train set, and used those learned categories to create the dummy variables in both train and test sets.

In [10]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data[['Sex', 'Embarked', 'Cabin']],  # predictors
    data['Survived'],  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((623, 3), (268, 3))

### Let's explore the cardinality

In [11]:
# sex has 2 labels

X_train['Sex'].unique()

array(['male', 'female'], dtype=object)

In [13]:
# embarked has 3 labels and missing data

X_train['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [12]:
# cabin has 9 labels and missing data

X_train['Cabin'].unique()

array(['E', 'D', nan, 'B', 'C', 'A', 'F', 'G', 'T'], dtype=object)

## One hot encoding with pandas

### Advantages

- quick
- returns pandas dataframe
- returns feature names for the dummy variables

### Limitations of pandas:

- it does not preserve information from train data to propagate to test data


-----

The pandas method get_dummies(), will create as many binary variables as categories in the variable:

If the variable colour has 3 categories in the train data, it will create 2 dummy variables. However, if the variable colour has 5 categories in the test data, it will create 4 binary variables, therefore train and test sets will end up with different number of features and will be incompatible with training and scoring using Scikit-learn.

In practice, we shouldn't be using get-dummies in our machine learning pipelines. It is however useful, for a quick data exploration. Let's look at this with examples.

### into k  dummy variables

In [0]:
# we can create dummy variables with the build in
# pandas method get_dummies

tmp = pd.get_dummies(X_train['Sex'])

tmp.head()

,female,male
501,1,0
588,1,0
402,1,0
1193,0,1
686,1,0


In [14]:
# for better visualisation let's put the dummies next
# to the original variable

pd.concat([X_train['Sex'],
           pd.get_dummies(X_train['Sex'])], axis=1).head()

,Sex,female,male
857,male,0,1
52,female,1,0
386,male,0,1
124,male,0,1
578,female,1,0


In [15]:
# and now let's repeat for embarked

tmp = pd.get_dummies(X_train['Embarked'])

tmp.head()

,C,Q,S
857,0,0,1
52,1,0,0
386,0,0,1
124,0,0,1
578,1,0,0


In [16]:
# for better visualisation

pd.concat([X_train['Embarked'],
           pd.get_dummies(X_train['Embarked'])], axis=1).head()

,Embarked,C,Q,S
857,S,0,0,1
52,C,1,0,0
386,S,0,0,1
124,S,0,0,1
578,C,1,0,0


In [17]:
# and now for cabin

tmp = pd.get_dummies(X_train['Cabin'])

tmp.head()

,A,B,C,D,E,F,G,T
857,0,0,0,0,1,0,0,0
52,0,0,0,1,0,0,0,0
386,0,0,0,0,0,0,0,0
124,0,0,0,1,0,0,0,0
578,0,0,0,0,0,0,0,0


In [18]:
# and now for all variables together: train set

tmp = pd.get_dummies(X_train)

print(tmp.shape)

tmp.head()

(623, 13)


,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T
857,0,1,0,0,1,0,0,0,0,1,0,0,0
52,1,0,1,0,0,0,0,0,1,0,0,0,0
386,0,1,0,0,1,0,0,0,0,0,0,0,0
124,0,1,0,0,1,0,0,0,1,0,0,0,0
578,1,0,1,0,0,0,0,0,0,0,0,0,0


In [19]:
# and now for all variables together: test set

tmp = pd.get_dummies(X_test)

print(tmp.shape)

tmp.head()

(268, 12)


,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G
495,0,1,1,0,0,0,0,0,0,0,0,0
648,0,1,0,0,1,0,0,0,0,0,0,0
278,0,1,0,1,0,0,0,0,0,0,0,0
31,1,0,1,0,0,0,1,0,0,0,0,0
255,1,0,1,0,0,0,0,0,0,0,0,0


Notice the positives of pandas get_dummies:
- dataframe returned with feature names

**And the limitations:**

The train set contains 13 dummy features, whereas the test set contains 12 features. This occurred because there was no category T in cabin in the test set.

This will cause problems if training and scoring models with scikit-learn, because predictors require train and test sets to be of the same shape.

### into k -1 

In [20]:
# obtaining k-1 labels: we need to indicate get_dummies
# to drop the first binary variable

tmp = pd.get_dummies(X_train['Sex'], drop_first=True)

tmp.head()

,male
857,1
52,0
386,1
124,1
578,0


In [21]:
# obtaining k-1 labels: we need to indicate get_dummies
# to drop the first binary variable

tmp = pd.get_dummies(X_train['Embarked'], drop_first=True)

tmp.head()

,Q,S
857,0,1
52,0,0
386,0,1
124,0,1
578,0,0


For embarked, if an observation shows 0 for Q and S, then its value must be C, the remaining category.

Caveat, this variable has missing data, so unless we encode missing data as well, all the information contained in the variable is not captured.

In [22]:
# altogether: train set

tmp = pd.get_dummies(X_train, drop_first=True)

print(tmp.shape)

tmp.head()

(623, 10)


,Sex_male,Embarked_Q,Embarked_S,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T
857,1,0,1,0,0,0,1,0,0,0
52,0,0,0,0,0,1,0,0,0,0
386,1,0,1,0,0,0,0,0,0,0
124,1,0,1,0,0,1,0,0,0,0
578,0,0,0,0,0,0,0,0,0,0


In [23]:
# altogether: test set

tmp = pd.get_dummies(X_test, drop_first=True)

print(tmp.shape)

tmp.head()

(268, 9)


,Sex_male,Embarked_Q,Embarked_S,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G
495,1,0,0,0,0,0,0,0,0
648,1,0,1,0,0,0,0,0,0
278,1,1,0,0,0,0,0,0,0
31,0,0,0,1,0,0,0,0,0
255,0,0,0,0,0,0,0,0,0


### Bonus: get_dummies() can handle missing values

In [26]:
# we can add an additional dummy variable to indicate
# missing data

pd.get_dummies(X_train['Embarked'], drop_first=True, dummy_na=True).head()

,Q,S,NaN
857,0,1,0
52,0,0,0
386,0,1,0
124,0,1,0
578,0,0,0


## One hot encoding with Scikit-learn

### Advantages

- quick
- Creates the same number of features in train and test set

### Limitations

- it returns a numpy array instead of a pandas dataframe
- it does not return the variable names, therefore inconvenient for variable exploration

In [32]:
# we create and train the encoder

encoder = OneHotEncoder(categories='auto',
                       drop='first', # to return k-1, use drop=false to return k dummies
                       sparse=False,
                      # handle_unknown='ignore') # helps deal with rare labels
                       handle_unknown='error') # helps deal with rare labels

encoder.fit(X_train.fillna('Missing'))

OneHotEncoder(categories='auto', drop='first', dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=False)

In [34]:
# we observe the learned categories

encoder.categories_

[array(['female', 'male'], dtype=object),
 array(['C', 'Missing', 'Q', 'S'], dtype=object),
 array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'Missing', 'T'], dtype=object)]

In [35]:
# transform the train set

tmp = encoder.transform(X_train.fillna('Missing'))

pd.DataFrame(tmp).head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


If we look at the categories, we can derive that the first column is 'male' and columns 1-3 are 'Missing', 'Q' and 'S', respectively. However, it does require quite a bit of manual work to add the feature names back into the data.

There are ways to do that, for example, creating 1 one hot encoder transformer per variable. See for example this post:
http://www.insightsbot.com/blog/McTKK/python-one-hot-encoding-with-scikit-learn

However, I am not a great fan of this implementation. I prefer to use feature-engine.

In [36]:
# we can go ahead and transfom the test set

tmp = encoder.transform(X_test.fillna('Missing'))

pd.DataFrame(tmp).head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


We can see that train and test contain the same number of features.

More details about Scikit-learn's OneHotEncoder can be found here:
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html


## One hot encoding with Feature-Engine

### Advantages
- quick
- returns dataframe
- returns feature names
- allows to select features to encode

### Limitations
- Not sure yet.

In [37]:
ohe_enc = OneHotCategoricalEncoder(
    top_categories=None,
    variables=['Sex', 'Embarked'], # we can select which variables to encode
    drop_last=True) # to return k-1, false to return k


ohe_enc.fit(X_train)

OneHotCategoricalEncoder(drop_last=True, top_categories=None,
                         variables=['Sex', 'Embarked'])

In [0]:
tmp = ohe_enc.transform(X_train)

tmp.head()

,cabin,sex_female,embarked_S,embarked_C,embarked_Q
501,NaN,1,1,0,0
588,NaN,1,1,0,0
402,NaN,1,0,1,0
1193,NaN,0,0,0,1
686,NaN,1,0,0,1


Note how feature-engine returns the dummy variables with their names, and drops the original variable, leaving the dataset ready for further exploration or building machine learning models.

In [38]:
tmp = ohe_enc.transform(X_test)

tmp.head()

,Cabin,Sex_male,Embarked_S,Embarked_C,Embarked_Q
495,NaN,1,0,1,0
648,NaN,1,1,0,0
278,NaN,1,0,0,1
31,B,0,0,1,0
255,NaN,0,0,1,0


In [39]:
# Feature-Engine's one hot encoder also selects
# all categorical variables automatically

ohe_enc = OneHotCategoricalEncoder(
    top_categories=None,
    drop_last=True) # to return k-1, false to return k


ohe_enc.fit(X_train)

OneHotCategoricalEncoder(drop_last=True, top_categories=None,
                         variables=['Sex', 'Embarked', 'Cabin'])

In [40]:
ohe_enc.variables

['Sex', 'Embarked', 'Cabin']

In [41]:
tmp = ohe_enc.transform(X_train)

tmp.head()

,Sex_male,Embarked_S,Embarked_C,Embarked_Q,Cabin_E,Cabin_D,Cabin_nan,Cabin_B,Cabin_C,Cabin_A,Cabin_F,Cabin_G
857,1,1,0,0,1,0,0,0,0,0,0,0
52,0,0,1,0,0,1,0,0,0,0,0,0
386,1,1,0,0,0,0,0,0,0,0,0,0
124,1,1,0,0,0,1,0,0,0,0,0,0
578,0,0,1,0,0,0,0,0,0,0,0,0


In [0]:
tmp = ohe_enc.transform(X_test)

tmp.head()

,sex_female,embarked_S,embarked_C,embarked_Q,cabin_nan,cabin_E,cabin_C,cabin_D,cabin_B,cabin_A,cabin_F,cabin_T
1139,0,1,0,0,0,0,0,0,0,0,0,0
533,1,1,0,0,0,0,0,0,0,0,0,0
459,0,1,0,0,0,0,0,0,0,0,0,0
1150,0,1,0,0,0,0,0,0,0,0,0,0
393,0,1,0,0,0,0,0,0,0,0,0,0


Note how this encoder returns a variable cabin_T for the test set as well, even though this category is not present in the test set. This allows the integration with Scikit-learn pipeline and scoring of test set by the built algorithm..

In fact, we can check that the sum of cabin_t is 0:

In [0]:
tmp['cabin_T'].sum()

0